<a href="https://colab.research.google.com/github/saktheeswaranswan/4-bit-alu-logi-sim-i-build/blob/main/yolo_dataset_openimages_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://storage.googleapis.com/openimages/2018_04/image_ids_and_rotation.csv

--2023-02-20 17:08:58--  https://storage.googleapis.com/openimages/2018_04/image_ids_and_rotation.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3348497077 (3.1G) [text/csv]
Saving to: ‘image_ids_and_rotation.csv’

image_ids_and_rotat 100%[===================>]   3.12G   232MB/s    in 15s     

2023-02-20 17:09:13 (216 MB/s) - ‘image_ids_and_rotation.csv’ saved [3348497077/3348497077]



In [3]:
!wget https://storage.googleapis.com/openimages/v7/oidv7-train-annotations-human-imagelabels.csv

--2023-02-20 17:09:30--  https://storage.googleapis.com/openimages/v7/oidv7-train-annotations-human-imagelabels.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 172.253.117.128, 142.250.99.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2735816020 (2.5G) [text/csv]
Saving to: ‘oidv7-train-annotations-human-imagelabels.csv’

oidv7-train-annotat 100%[===================>]   2.55G   171MB/s    in 17s     

2023-02-20 17:09:47 (155 MB/s) - ‘oidv7-train-annotations-human-imagelabels.csv’ saved [2735816020/2735816020]



In [11]:
!wget https://storage.googleapis.com/openimages/v7/oidv7-class-descriptions-boxable.csv

--2023-02-20 17:20:57--  https://storage.googleapis.com/openimages/v7/oidv7-class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.135.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12064 (12K) [text/csv]
Saving to: ‘oidv7-class-descriptions-boxable.csv’

oidv7-class-descrip 100%[===================>]  11.78K  --.-KB/s    in 0s      

2023-02-20 17:20:57 (73.5 MB/s) - ‘oidv7-class-descriptions-boxable.csv’ saved [12064/12064]



In [15]:
import csv
import subprocess
import os
import multiprocessing


def download(line):

    lineParts = line.split(',')
    print("downloading {}...".format(lineParts[0]))
    subprocess.run([ 'aws', 's3', '--no-sign-request', '--only-show-errors', 'cp', 's3://open-images-dataset/'+runMode+'/'+lineParts[0]+".jpg", 'JPEGImages/'+lineParts[0]+".jpg"])
    with open('labels/%s.txt'%(lineParts[0]),'a') as f:
        f.write(' '.join([str(ind),str((float(lineParts[5]) + float(lineParts[4]))/2), str((float(lineParts[7]) + float(lineParts[6]))/2), str(float(lineParts[5])-float(lineParts[4])),str(float(lineParts[7])-float(lineParts[6]))])+'\n')


runMode = "train"
classes = ["Car"]

with open('/content/oidv7-class-descriptions-boxable.csv', mode='r') as infile:
    reader = csv.reader(infile)
    dict_list = {rows[1]:rows[0] for rows in reader}

# Commented out to enable resuming the download later
# subprocess.run(['rm', '-rf', 'JPEGImages'])
# subprocess.run([ 'mkdir', 'JPEGImages'])

# subprocess.run(['rm', '-rf', 'labels'])
# subprocess.run([ 'mkdir', 'labels'])

# This variable enables pausing/continuing download
already_down = 0

for ind in range(0, len(classes)):
    
    className = classes[ind]
    print("Class " + str(ind) + " : " + className)

    commandStr = "grep " + dict_list[className] + " " + runMode + "-annotations-bbox.csv"
    print(commandStr)
    class_annotations = subprocess.run(commandStr.split(), stdout=subprocess.PIPE).stdout.decode('utf-8')
    class_annotations = class_annotations.splitlines()

    totalNumOfAnnotations = len(class_annotations)
    print("Total number of annotations : "+str(totalNumOfAnnotations))

    pool = multiprocessing.Pool()
    pool.map(download, class_annotations[already_down:totalNumOfAnnotations], 1000)

Class 0 : Car
grep /m/0k4j train-annotations-bbox.csv
Total number of annotations : 0


In [6]:
!wget https://storage.googleapis.com/openimages/v7/oidv7-class-descriptions.csv

--2023-02-20 17:14:58--  https://storage.googleapis.com/openimages/v7/oidv7-class-descriptions.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.135.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 501178 (489K) [text/csv]
Saving to: ‘oidv7-class-descriptions.csv’

oidv7-class-descrip 100%[===================>] 489.43K  --.-KB/s    in 0.005s  

2023-02-20 17:14:58 (101 MB/s) - ‘oidv7-class-descriptions.csv’ saved [501178/501178]



In [9]:
!wget https://raw.githubusercontent.com/openimages/dataset/master/downloader.py


--2023-02-20 17:19:12--  https://raw.githubusercontent.com/openimages/dataset/master/downloader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4244 (4.1K) [text/plain]
Saving to: ‘downloader.py’

downloader.py       100%[===================>]   4.14K  --.-KB/s    in 0s      

2023-02-20 17:19:12 (52.5 MB/s) - ‘downloader.py’ saved [4244/4244]



In [10]:
!python downloader.py $IMAGE_LIST_FILE --download_folder=$DOWNLOAD_FOLDER --num_processes=5


Traceback (most recent call last):
  File "downloader.py", line 40, in <module>
    import boto3
ModuleNotFoundError: No module named 'boto3'
